# NEDLBF (OMIM:620489)
[Neurodevelopmental disorder with dysmorphic facies and behavioral abnormalities](https://omim.org/entry/620489) 	 

Data from [Jia X, et al De novo variants in genes regulating stress granule assembly associate with neurodevelopmental disorders. Sci Adv. 2022 Aug 19;8(33):eabo7112. doi: 10.1126/sciadv.abo7112. Epub 2022 Aug 17. PMID: 35977029; PMCID: PMC9385150.](https://pubmed.ncbi.nlm.nih.gov/35977029/).

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from IPython.display import HTML, display
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import *
import pyphetools
print(f"pyphetools version {pyphetools.__version__}")

pyphetools version 0.9.56


In [2]:
PMID = "PMID:35977029"
title = "De novo variants in genes regulating stress granule assembly associate with neurodevelopmental disorders"
cite = Citation(pmid=PMID, title=title)
parser = HpoParser(hpo_json_file="../hp.json")
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
metadata = MetaData(created_by="ORCID:0000-0002-5648-2155", citation=cite)
metadata.default_versions_with_hpo(version=hpo_version)
print(f"HPO version {hpo_version}")

HPO version 2024-02-27


In [3]:
df = pd.read_excel('input/UBAP2L_Jia_2022.xlsx')
dft = df.transpose()
dft.columns = dft.iloc[0]
dft.drop(dft.index[0], inplace=True)
# simplify the name of the id column to remove e.g.,(11.3 this manuscript) f
dft.set_index("Case ID", inplace = True)
dft["individual_id"] = dft.index
dft.head(2)

Family Index,Region,"Country, ethnicity information",NM_014847.4,UBAP2L Variants (NM_014847.4)(GRCh37),Variant Detection Method(s),Mutation inheritance,Age at clinical diagnosis(y/m),Age at last examination(y/m),Gender,Delivery - weeks gestation,...,Gastrointestinal disturbance,Skeletal alterations,Visual impairment,Hearing impairment,Hypotonia,Facial deformity (description),Hands,Feet,Other symptoms not mentioned above,individual_id
Case ID,,,,,,,,,,,,,,,,,,,,,
20190W0028,"Guangxi, China","China, Asian",c.88C>T,"chr1:154197687C>T,c.88C>T,p.Q30*",WES (Trio),De Novo,2y3m,3y9m,male,36w+1,...,-,+(Mild strephenopodia),"+(Congenital abnormal eye development(nystagmus, leukoma,esotropia);)",-,-,"Round face, Asymmetrical palpebral fissure, Cup ears, Low front hairline, Deviated mouth, Deep and prominent concha","Digital pudds, Clinodactyly V finger",-,ND,20190W0028
A43,"Sichuan, China","China, Asian",c.370C>T,"chr1:g.154207157C>T,c.370C>T,p.R124*",WES (Trio),De Novo,9y,9y,male,ND,...,-,-,-,-,-,ND,-,-,ND,A43


In [4]:
column_mapper_list = list()

In [5]:
#res = OptionColumnMapper.autoformat(df=dft, hpo_cr=hpo_cr)
#print(res)

In [6]:
def iso8601(age):
    if age == "ND":
        return "na"
    age = f"P{age.upper()}"
    return age
    
dft["Age at clinical diagnosis(y/m)"]
dft["age_of_onset"] = dft["Age at clinical diagnosis(y/m)"].apply(lambda x:iso8601(x))
ageOfOnsetMapper = AgeColumnMapper.iso8601(column_name="age_of_onset")
#ageOfOnsetMapper.preview_column(dft)
dft["age_at_last_encounter"] = dft["Age at last examination(y/m)"].apply(lambda x:iso8601(x))
ageLastEncounterMapper = AgeColumnMapper.iso8601(column_name="age_at_last_encounter")
#ageLastEncounterMapper.preview_column(dft)
sexMapper = SexColumnMapper(column_name="Gender", male_symbol="male", female_symbol="female")
#sexMapper.preview_column(dft)

In [7]:
prenatal_d = {'IUGR/SGA/Premature/short femur(Prenatal ultrasound)': ["Intrauterine growth retardation",'Short femur'],
 'Breech': 'Breech presentation',
 'IUGR/Premature/Anhydramnios after AROM with CAF': ["Intrauterine growth retardation",'Anhydramnios']}
excluded = {}
prenatalMapper = OptionColumnMapper(column_name="Prenatal history", concept_recognizer=hpo_cr, option_d=prenatal_d, excluded_d=excluded)
column_mapper_list.append(prenatalMapper)
prenatalMapper.preview_column(dft)

,mapping,count
0,Intrauterine growth retardation (HP:0001511) (observed),2
1,Short femur (HP:0003097) (observed),1
2,Tremor (HP:0001337) (observed),1
3,Breech presentation (HP:0001623) (observed),1
4,Anhydramnios (HP:0025700) (observed),1


In [8]:
feeding = SimpleColumnMapper(column_name="Infant feeding difficulties", hpo_id="HP:0011968", hpo_label="Feeding difficulties", observed="+", excluded="-")
column_mapper_list.append(feeding)
feeding.preview_column(dft)

,mapping,count
0,"original value: ""+"" -> HP: Feeding difficulties (HP:0011968) (observed)",8
1,"original value: ""ND"" -> HP: Feeding difficulties (HP:0011968) (not measured)",1
2,"original value: ""-"" -> HP: Feeding difficulties (HP:0011968) (excluded)",3


In [9]:
ftt = SimpleColumnMapper(column_name="Failure to thrive", hpo_id="HP:0001508", hpo_label="Failure to thrive", observed="+", excluded="-")
column_mapper_list.append(ftt)
ftt.preview_column(dft)

,mapping,count
0,"original value: ""+"" -> HP: Failure to thrive (HP:0001508) (observed)",4
1,"original value: ""ND"" -> HP: Failure to thrive (HP:0001508) (not measured)",4
2,"original value: ""-"" -> HP: Failure to thrive (HP:0001508) (excluded)",4


In [10]:
asd_d = {
 '+(ADOS/ADI-R)': 'Autistic behavior',
 '+(ADI-R/ADOS)': 'Autistic behavior',
 '+(DSM-5)': 'Autistic behavior'}
excluded = {'-(ADOS:4)': 'Autistic behavior','-(CARS-33 scores/ABC-46 scores)': 'Autistic behavior',}
asdMapper = OptionColumnMapper(column_name="Autism spectrum disorder (criteria)", concept_recognizer=hpo_cr, option_d=asd_d, excluded_d=excluded)
column_mapper_list.append(asdMapper)
asdMapper.preview_column(dft)

,mapping,count
0,Autistic behavior (HP:0000729) (excluded),2
1,Autistic behavior (HP:0000729) (observed),3


In [11]:
language_d = {'+ ( Early Language Milestone Scale(3Y9M):Phonological and Verbal expression:10 scores(13 months); Auditory perception and Comprehension:11 scores(14 months); Visual perception and Understanding:12 scores(12-13 months); Full scale:33 scores(13-14 months);)': 'Delayed speech and language development',
 '+': 'Delayed speech and language development',
 '+(First words at 1yr.; 2-3 words at 3yrs.)': 'Delayed speech and language development',
 '+(Receptive language test 75/80)': 'Delayed speech and language development',
 '+(No English and Franch)': 'Delayed speech and language development',
 '+(Talked at 4 yrs.)': 'Delayed speech and language development',
 '+(Stuttering)': 'Stuttering',
 '+(First words at 2.5 yrs.)': 'Delayed speech and language development',
 '+(Marked speech delay)': 'Delayed speech and language development',
 '+(~60 words in vocabulary)': 'Delayed speech and language development'}
excluded = {}
languageMapper = OptionColumnMapper(column_name="Speech-language problems", concept_recognizer=hpo_cr, option_d=language_d, excluded_d=excluded)
column_mapper_list.append(languageMapper)
languageMapper.preview_column(dft)

,mapping,count
0,Delayed speech and language development (HP:0000750) (observed),10
1,Stuttering (HP:0025268) (observed),1


In [12]:
motor_delay_d = {'+(Mild delayed in fine and gross motor skills; Sitting at 10 months; walking at 15\n  months; going up and down stairs at >2yrs.; writing at 4yrs.;)': 'Motor delay',
 '+(Walking at 18 months)': 'Motor delay',
 '+(Walked at 15 months)': 'Motor delay',
 '+(Age of sitting:12 months; Age of standing:17 months; Age of walking without\n  support:22 months;)': 'Motor delay',
 '+(Marked motor delay)': 'Motor delay'}
excluded = {}
motor_delayMapper = OptionColumnMapper(column_name="Motor delay", concept_recognizer=hpo_cr, option_d=motor_delay_d, excluded_d=excluded)
column_mapper_list.append(motor_delayMapper)
motor_delayMapper.preview_column(dft)

,mapping,count
0,Motor delay (HP:0001270) (observed),3


In [13]:
intellectual_disability_d = {
    "+":'Intellectual disability',
 '+(Mild)': 'Intellectual disability',
 '+(Functioning at the level of 9-10 years old in most areas (at age of 15 yrs.))': 'Intellectual disability'}
excluded = { "-":'Intellectual disability',}
intellectual_disabilityMapper = OptionColumnMapper(column_name="Intellectual disability", concept_recognizer=hpo_cr, option_d=intellectual_disability_d, excluded_d=excluded)
column_mapper_list.append(intellectual_disabilityMapper)
intellectual_disabilityMapper.preview_column(dft)

,mapping,count
0,Intellectual disability (HP:0001249) (observed),8
1,Intellectual disability (HP:0001249) (excluded),2


In [14]:
regression_of_developmental_milestones_d = {"+":"Developmental regression",
 '+(Speech regression)': 'Developmental regression'}
excluded = {"-":"Developmental regression",}
regression_of_developmental_milestonesMapper = OptionColumnMapper(column_name="Regression of developmental milestones", concept_recognizer=hpo_cr, option_d=regression_of_developmental_milestones_d, excluded_d=excluded)
column_mapper_list.append(regression_of_developmental_milestonesMapper)
regression_of_developmental_milestonesMapper.preview_column(dft)

,mapping,count
0,Developmental regression (HP:0002376) (excluded),9
1,Developmental regression (HP:0002376) (observed),1


In [15]:

developmental_d = {"+(GDD)(SYMBOLIC PLAY TEST<12 months)(The Infants-Junior Middle School Students' Social-Life Abilities Scales (SM scales)": 'Global developmental delay',
 'SM score=9; marginal defect;)': 'Global developmental delay',
 '+(Mild DD)': 'Global developmental delay',
 '+(DD)': 'Global developmental delay',
 '+': 'Global developmental delay',
 '+(Poor effort tolerance)': 'Global developmental delay',
 '+(Delayed closing of the anterior fontanelle)': 'Global developmental delay'}
excluded = {'-': 'Global developmental delay',}
developmentalMapper = OptionColumnMapper(column_name="other developmental problems?", concept_recognizer=hpo_cr, option_d=developmental_d, excluded_d=excluded)
column_mapper_list.append(developmentalMapper)
developmentalMapper.preview_column(dft)


,mapping,count
0,Global developmental delay (HP:0001263) (observed),6
1,Global developmental delay (HP:0001263) (excluded),2


In [16]:
anxiety = SimpleColumnMapper(column_name="Anxiety (type)", hpo_id="HP:0000739", hpo_label="Anxiety", observed="+", excluded="-")
column_mapper_list.append(anxiety)
anxiety.preview_column(dft)

,mapping,count
0,"original value: ""-"" -> HP: Anxiety (HP:0000739) (excluded)",6
1,"original value: ""ND"" -> HP: Anxiety (HP:0000739) (not measured)",3
2,"original value: ""+"" -> HP: Anxiety (HP:0000739) (observed)",3


In [17]:
anxiety = SimpleColumnMapper(column_name="Anxiety (type)", hpo_id="HP:0007018", hpo_label="Attention deficit hyperactivity disorder", observed="+", excluded="-")
column_mapper_list.append(anxiety)
anxiety.preview_column(dft)

,mapping,count
0,"original value: ""-"" -> HP: Attention deficit hyperactivity disorder (HP:0007018) (excluded)",6
1,"original value: ""ND"" -> HP: Attention deficit hyperactivity disorder (HP:0007018) (not measured)",3
2,"original value: ""+"" -> HP: Attention deficit hyperactivity disorder (HP:0007018) (observed)",3


In [18]:
repetitive_behavior_d = {'+': 'Abnormal repetitive mannerisms',
 '+(Echolalia)': 'Echolalia'}
excluded = {'-': 'Abnormal repetitive mannerisms',}
repetitive_behaviorMapper = OptionColumnMapper(column_name="Repetitive behavior", concept_recognizer=hpo_cr, option_d=repetitive_behavior_d, excluded_d=excluded)
column_mapper_list.append(repetitive_behaviorMapper)
repetitive_behaviorMapper.preview_column(dft)

,mapping,count
0,Abnormal repetitive mannerisms (HP:0000733) (excluded),6
1,Abnormal repetitive mannerisms (HP:0000733) (observed),3
2,Echolalia (HP:0010529) (observed),1


In [19]:
#Compulsive behaviors HP:0000722
ocd = SimpleColumnMapper(column_name="Anxiety (type)", hpo_id="HP:0000722", hpo_label="Compulsive behaviors", observed="+", excluded="-")
column_mapper_list.append(ocd)
ocd.preview_column(dft)

,mapping,count
0,"original value: ""-"" -> HP: Compulsive behaviors (HP:0000722) (excluded)",6
1,"original value: ""ND"" -> HP: Compulsive behaviors (HP:0000722) (not measured)",3
2,"original value: ""+"" -> HP: Compulsive behaviors (HP:0000722) (observed)",3


In [20]:
dft["Aggressive behavior"]

Case ID
20190W0028        -
A43               -
family 3         ND
family 4          +
NA127             -
family 6          -
82847             -
14401.p1          +
family 9          -
family 19         -
DI1750_167397     +
family 12        ND
Name: Aggressive behavior, dtype: object

In [21]:
aggressive_behavior = SimpleColumnMapper(column_name="Anxiety (type)", hpo_id="HP:0000718", hpo_label="Aggressive behavior", observed="+", excluded="-")
column_mapper_list.append(aggressive_behavior)
aggressive_behavior.preview_column(dft)

,mapping,count
0,"original value: ""-"" -> HP: Aggressive behavior (HP:0000718) (excluded)",6
1,"original value: ""ND"" -> HP: Aggressive behavior (HP:0000718) (not measured)",3
2,"original value: ""+"" -> HP: Aggressive behavior (HP:0000718) (observed)",3


In [22]:
si_behavior = SimpleColumnMapper(column_name="Anxiety (type)", hpo_id="HP:0100716", hpo_label="Self-injurious behavior", observed="+", excluded="-")
column_mapper_list.append(si_behavior)
si_behavior.preview_column(dft)

,mapping,count
0,"original value: ""-"" -> HP: Self-injurious behavior (HP:0100716) (excluded)",6
1,"original value: ""ND"" -> HP: Self-injurious behavior (HP:0100716) (not measured)",3
2,"original value: ""+"" -> HP: Self-injurious behavior (HP:0100716) (observed)",3


In [23]:
epilepsy_d = {'+': 'Seizure',
 '+(Symptomatic generalized epilepsy: non-convulsive absence seizures)': 'Seizure'}
excluded = {'-': 'Seizure',}
epilepsy_Mapper = OptionColumnMapper(column_name="Epilepsy (specified in details)", concept_recognizer=hpo_cr, option_d=epilepsy_d, excluded_d=excluded)
column_mapper_list.append(epilepsy_Mapper)
epilepsy_Mapper.preview_column(dft)


,mapping,count
0,Seizure (HP:0001250) (excluded),7
1,Seizure (HP:0001250) (observed),3


In [24]:
sleep_disturbances_d = {
 '+(Sleep apnea diagnosed at 10 months and requiring nasal CPAP)': 'Sleep apnea',
 '+(Daytime sleepiness)': 'Drowsiness'}
excluded = {}
sleep_disturbancesMapper = OptionColumnMapper(column_name="Sleep disturbances", concept_recognizer=hpo_cr, option_d=sleep_disturbances_d, excluded_d=excluded)
column_mapper_list.append(sleep_disturbancesMapper)
sleep_disturbancesMapper.preview_column(dft)

,mapping,count
0,Sleep apnea (HP:0010535) (observed),1
1,Drowsiness (HP:0002329) (observed),1


In [25]:
other_neurological_problem_d = {'+(Oromandibular dystonia)': 'Oromandibular dystonia',
 '+(Metopic craniosynostosis & mild trigonocephaly)': 'Metopic synostosis',
 '+(Hyporeflexia)': 'Hyporeflexia'}
excluded = {}
other_neurological_problemMapper = OptionColumnMapper(column_name="Other neurological problem", concept_recognizer=hpo_cr, option_d=other_neurological_problem_d, excluded_d=excluded)
column_mapper_list.append(other_neurological_problemMapper)
other_neurological_problemMapper.preview_column(dft)

,mapping,count
0,Oromandibular dystonia (HP:0012048) (observed),1
1,Metopic synostosis (HP:0011330) (observed),1
2,Hyporeflexia (HP:0001265) (observed),1


In [26]:
shortstature = SimpleColumnMapper(column_name="Anxiety (type)", hpo_id="HP:0004322", hpo_label="Short stature", observed="+", excluded="-")
column_mapper_list.append(shortstature)
shortstature.preview_column(dft)

,mapping,count
0,"original value: ""-"" -> HP: Short stature (HP:0004322) (excluded)",6
1,"original value: ""ND"" -> HP: Short stature (HP:0004322) (not measured)",3
2,"original value: ""+"" -> HP: Short stature (HP:0004322) (observed)",3


In [27]:
skeletal_alterations_d = {
 '+(Lower limb skeletal anomalies(metatarsal adducts': 'Metatarsus adductus',
 'femoral anteversion': 'Increased femoral anteversion',
 'tibia torsion);)': 'Tibial torsion',
 '+(Brachydactyly; joint stiffness (hands': 'Brachydactyly',
 'and feet); symphalangism of thumb;)': 'Symphalangism of the thumb',
 '+(Scoliosis(14 degrees) and kyphosis(42 degrees))': 'Kyphoscoliosis',}
excluded = {}
skeletal_alterationsMapper = OptionColumnMapper(column_name="Skeletal alterations", concept_recognizer=hpo_cr, option_d=skeletal_alterations_d, excluded_d=excluded)
column_mapper_list.append(skeletal_alterationsMapper)
skeletal_alterationsMapper.preview_column(dft)

,mapping,count
0,Metatarsus adductus (HP:0001840) (observed),1
1,Increased femoral anteversion (HP:0012427) (observed),1
2,Brachydactyly (HP:0001156) (observed),1
3,Joint stiffness (HP:0001387) (observed),1
4,Kyphoscoliosis (HP:0002751) (observed),1


In [28]:
visual_impairment_d = {'+(Congenital abnormal eye development(nystagmus': 'Nystagmus',
 'esotropia);)': 'Esotropia',
 '+(Mild hypermetropic astigmatism)': 'Astigmatism',
 '+(Hypermetropia': 'Hypermetropia',
 '+(Myopia  (-2 diopter': 'Myopia',
                       "+":"Visual impairment",
 '+(Exotropia in right eye)': 'Exotropia'}
excluded = { "-":"Visual impairment",}
visual_impairmentMapper = OptionColumnMapper(column_name="Visual impairment", concept_recognizer=hpo_cr, option_d=visual_impairment_d, excluded_d=excluded)
column_mapper_list.append(visual_impairmentMapper)
visual_impairmentMapper.preview_column(dft)

,mapping,count
0,Nystagmus (HP:0000639) (observed),1
1,Esotropia (HP:0000565) (observed),1
2,Visual impairment (HP:0000505) (excluded),4
3,Astigmatism (HP:0000483) (observed),1
4,Hypermetropia (HP:0000540) (observed),1
5,Visual impairment (HP:0000505) (observed),1
6,Myopia (HP:0000545) (observed),1
7,Exotropia (HP:0000577) (observed),1


In [29]:
hearing_impairment_d = {'+': 'Hearing impairment',
 '+(Hyperacousia)': 'Hyperacusis'}
excluded = {'-': 'Hearing impairment',}
hearing_impairmentMapper = OptionColumnMapper(column_name="Hearing impairment", concept_recognizer=hpo_cr, option_d=hearing_impairment_d, excluded_d=excluded)
column_mapper_list.append(hearing_impairmentMapper)
hearing_impairmentMapper.preview_column(dft)

,mapping,count
0,Hearing impairment (HP:0000365) (excluded),8
1,Hyperacusis (HP:0010780) (observed),1


In [30]:
hypotonia = SimpleColumnMapper(column_name="Hypotonia", hpo_id="HP:0001252", hpo_label="Hypotonia", observed="+", excluded="-")
column_mapper_list.append(hypotonia)
hypotonia.preview_column(dft)

,mapping,count
0,"original value: ""-"" -> HP: Hypotonia (HP:0001252) (excluded)",4
1,"original value: ""+"" -> HP: Hypotonia (HP:0001252) (observed)",4
2,"original value: ""ND"" -> HP: Hypotonia (HP:0001252) (not measured)",4


In [31]:
facial_d = {'Round face': 'Round face',
 'Cup ears': 'Cupped ear',
 'Low front hairline': 'Low anterior hairline',
 'Mild synophrys': 'Synophrys',
 'Upslanting palpebral fissures': 'Upslanted palpebral fissure',
 'Hypertelorism': 'Hypertelorism',
 'Broad nasal bridge': 'Wide nasal bridge',
 'Ears small helix (scrumpled)': 'Crumpled ear',
 'Long face with facial asymmetry': 'Long face',
 'Thin upper lip': 'Thin upper lip vermilion',
 'Mild eversion of lower vermillion': 'Everted lower lip vermilion',
 'High-arched palate': 'High palate',
 'Prominent chin': 'Mandibular prognathia',
 'Small ears': 'Microtia',
 'Broad forehead': 'Broad forehead',
 'Small palpebral fissure': 'Narrow palpebral fissure',
 "Cow's lick on forehead": 'Frontal upsweep of hair',
 'Flat face': 'Flat face',
 'Deep set eyes': 'Deeply set eye',
 'Bulbous nose': 'Bulbous nose',
 'Long philtrum': 'Long philtrum',
 'Low-set and posteriorly rotated ears': 'Posteriorly rotated ears',
 'Epicanthus': 'Epicanthus',
 'Medial eyebrow flare': 'Medial flaring of the eyebrow',
 'Wide nasal root': 'Wide nasal bridge',
 'Depressed nasal tip': 'Depressed nasal tip',
 'High broad forehead': 'Broad forehead'}
excluded = {}
facialMapper = OptionColumnMapper(column_name="Facial deformity (description)", concept_recognizer=hpo_cr, option_d=facial_d, excluded_d=excluded)
column_mapper_list.append(facialMapper)
facialMapper.preview_column(dft)

,mapping,count
0,Round face (HP:0000311) (observed),1
1,Cupped ear (HP:0000378) (observed),1
2,Low anterior hairline (HP:0000294) (observed),1
3,Synophrys (HP:0000664) (observed),2
4,Upslanted palpebral fissure (HP:0000582) (observed),1
5,Hypertelorism (HP:0000316) (observed),3
6,Wide nasal bridge (HP:0000431) (observed),2
7,Crumpled ear (HP:0009901) (observed),1
8,Long face (HP:0000276) (observed),1
9,Thin upper lip vermilion (HP:0000219) (observed),2


In [32]:
hands_d = {
 'Clinodactyly V finger': 'Clinodactyly',
 'symphalangism of thumbs': 'Symphalangism of the thumb',
 'brachydactyly': 'Brachydactyly',
 'Atrophy thenar and hypothenar eminence': 'Thenar muscle atrophy',
}
excluded = {}
handsMapper = OptionColumnMapper(column_name="Hands", concept_recognizer=hpo_cr, option_d=hands_d, excluded_d=excluded)
column_mapper_list.append(handsMapper)
handsMapper.preview_column(dft)



,mapping,count
0,Clinodactyly (HP:0030084) (observed),1
1,Symphalangism of the thumb (HP:0009656) (observed),1
2,Brachydactyly (HP:0001156) (observed),1
3,Thenar muscle atrophy (HP:0003393) (observed),1


In [33]:
feet_d = {
 'Nail hypoplasia of hallux': 'Hypoplastic toenails',
 'Flat feet': 'Pes planus',
 'Nails hypoplasia': 'Small nail',
 'Pointing finger joint contracture': 'Finger joint contracture',
 'Broad halluces': 'Broad hallux',
 'brachydactyly of the toes': 'Short toe'}
excluded = {}
feetMapper = OptionColumnMapper(column_name="Feet", concept_recognizer=hpo_cr, option_d=feet_d, excluded_d=excluded)
column_mapper_list.append(feetMapper)
feetMapper.preview_column(dft)

,mapping,count
0,Hypoplastic toenails (HP:0001800) (observed),1
1,Pes planus (HP:0001763) (observed),1
2,Small nail (HP:0001792) (observed),1
3,Finger joint contracture (HP:0034681) (observed),1
4,Broad hallux (HP:0010055) (observed),1
5,Short toe (HP:0001831) (observed),1


In [34]:
other_symptoms_not_mentioned_above_d = {
 'Hypertrichosis': 'Hypertrichosis',
 'Three cafe-au-lait spots and one hypochromic macula': 'Cafe-au-lait spot',
 'Respiratory problems(including chronic bronchitis)': 'Chronic bronchitis',
 'Recurrent urinary tract infections': 'Recurrent urinary tract infections',
 'Asthma': 'Asthma',
 'Cutis  marmorata': 'Cutis marmorata'}
excluded = {}
otherMapper = OptionColumnMapper(column_name="Other symptoms not mentioned above", concept_recognizer=hpo_cr, option_d=other_symptoms_not_mentioned_above_d, excluded_d=excluded)
column_mapper_list.append(otherMapper)
otherMapper.preview_column(dft)

,mapping,count
0,Hypertrichosis (HP:0000998) (observed),1
1,Cafe-au-lait spot (HP:0000957) (observed),1
2,Chronic bronchitis (HP:0004469) (observed),1
3,Recurrent urinary tract infections (HP:0000010) (observed),1
4,Asthma (HP:0002099) (observed),1
5,Cutis marmorata (HP:0000965) (observed),1


In [35]:
UBAP2L_transcript='NM_014847.4'
vmanager = VariantManager(df=dft, individual_column_name="individual_id",gene_symbol="UBAP2L",transcript=UBAP2L_transcript,
                          allele_1_column_name="NM_014847.4")

In [36]:
vmanager.to_summary()

,status,count,alleles
0,mapped,12,"c.562G>T, c.2724C>A, c.3168+3A>G, c.1965del, c.1964dup, c.370C>T, c.703+3dup, c.590+1G>A, c.1714C>T, c.2158_2165del, c.1846C>T, c.88C>T"
1,unmapped,0,


In [37]:
variantMapper = VariantColumnMapper(variant_column_name='NM_014847.4', 
                                    variant_d=vmanager.get_variant_d(), 
                                    default_genotype="heterozygous")

In [38]:
NEDLBF = Disease(disease_id='OMIM:620494', disease_label='Neurodevelopmental disorder with impaired language, behavioral abnormalities, and dysmorphic facies')

encoder = CohortEncoder(df=dft, 
                        hpo_cr=hpo_cr, 
                        column_mapper_list=column_mapper_list, 
                        individual_column_name="individual_id", 
                        age_of_onset_mapper=ageOfOnsetMapper,
                        age_at_last_encounter_mapper=ageLastEncounterMapper,
                        sexmapper=sexMapper,
                        variant_mapper=variantMapper,
                        metadata=metadata)
encoder.set_disease(NEDLBF)

In [39]:
individuals = encoder.get_individuals()

Could not parse the following as ISO8601 ages: na (n=1)


In [40]:
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1, allelic_requirement=AllelicRequirement.MONO_ALLELIC)
validated_individuals = cvalidator.get_validated_individual_list()
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

Level,Error category,Count
WARNING,REDUNDANT,1
INFORMATION,NOT_MEASURED,27


In [41]:
individuals = cvalidator.get_error_free_individual_list()

In [42]:
table = PhenopacketTable(individual_list=individuals, metadata=metadata)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
20190W0028 (MALE; P3Y9M),"Neurodevelopmental disorder with impaired language, behavioral abnormalities, and dysmorphic facies (OMIM:620494)",NM_014847.4:c.88C>T (heterozygous),Intrauterine growth retardation (HP:0001511); Short femur (HP:0003097); Feeding difficulties (HP:0011968); Failure to thrive (HP:0001508); Delayed speech and language development (HP:0000750); Intellectual disability (HP:0001249); Global developmental delay (HP:0001263); Nystagmus (HP:0000639); Esotropia (HP:0000565); Round face (HP:0000311); Cupped ear (HP:0000378); Low anterior hairline (HP:0000294); Clinodactyly (HP:0030084); excluded: Autistic behavior (HP:0000729); excluded: Developmental regression (HP:0002376); excluded: Anxiety (HP:0000739); excluded: Attention deficit hyperactivity disorder (HP:0007018); excluded: Abnormal repetitive mannerisms (HP:0000733); excluded: Compulsive behaviors (HP:0000722); excluded: Aggressive behavior (HP:0000718); excluded: Self-injurious behavior (HP:0100716); excluded: Seizure (HP:0001250); excluded: Short stature (HP:0004322); excluded: Hearing impairment (HP:0000365); excluded: Hypotonia (HP:0001252)
A43 (MALE; P9Y),"Neurodevelopmental disorder with impaired language, behavioral abnormalities, and dysmorphic facies (OMIM:620494)",NM_014847.4:c.370C>T (heterozygous),Autistic behavior (HP:0000729); Intellectual disability (HP:0001249); Abnormal repetitive mannerisms (HP:0000733); excluded: Developmental regression (HP:0002376); excluded: Global developmental delay (HP:0001263); excluded: Seizure (HP:0001250); excluded: Visual impairment (HP:0000505); excluded: Hearing impairment (HP:0000365); excluded: Hypotonia (HP:0001252)
family 3 (FEMALE; ),"Neurodevelopmental disorder with impaired language, behavioral abnormalities, and dysmorphic facies (OMIM:620494)",NM_014847.4:c.562G>T (heterozygous),Tremor (HP:0001337); Feeding difficulties (HP:0011968); Delayed speech and language development (HP:0000750); Global developmental delay (HP:0001263); Metatarsus adductus (HP:0001840); Increased femoral anteversion (HP:0012427); Hypotonia (HP:0001252)
family 4 (FEMALE; P7Y),"Neurodevelopmental disorder with impaired language, behavioral abnormalities, and dysmorphic facies (OMIM:620494)",NM_014847.4:c.590+1G>A (heterozygous),Astigmatism (HP:0000483); Feeding difficulties (HP:0011968); Symphalangism of the thumb (HP:0009656); Joint stiffness (HP:0001387); Global developmental delay (HP:0001263); Synophrys (HP:0000664); Upslanted palpebral fissure (HP:0000582); Echolalia (HP:0010529); Brachydactyly (HP:0001156); Hypoplastic toenails (HP:0001800); Failure to thrive (HP:0001508); Delayed speech and language development (HP:0000750); excluded: Autistic behavior (HP:0000729); excluded: Intellectual disability (HP:0001249); excluded: Developmental regression (HP:0002376); excluded: Anxiety (HP:0000739); excluded: Attention deficit hyperactivity disorder (HP:0007018); excluded: Compulsive behaviors (HP:0000722); excluded: Aggressive behavior (HP:0000718); excluded: Self-injurious behavior (HP:0100716); excluded: Seizure (HP:0001250); excluded: Short stature (HP:0004322); excluded: Hearing impairment (HP:0000365); excluded: Hypotonia (HP:0001252)
NA127 (FEMALE; P15Y8M),"Neurodevelopmental disorder with impaired language, behavioral abnormalities, and dysmorphic facies (OMIM:620494)",NM_014847.4:c.703+3dup (heterozygous),Delayed speech and language development (HP:0000750); Intellectual disability (HP:0001249); Abnormal repetitive mannerisms (HP:0000733); Seizure (HP:0001250); Oromandibular dystonia (HP:0012048); Synophrys (HP:0000664); Cafe-au-lait spot (HP:0000957); excluded: Feeding difficulties (HP:0011968); excluded: Failure to thrive (HP:0001508); excluded: Developmental regression (HP:0002376); excluded: Global developmental delay (HP:0001263); excluded: Anxiety (HP:0000739); excluded: Attention deficit hyperactivity disorder (HP:0007018); excluded: Compulsive behaviors (HP:0000

In [43]:
Individual.output_individuals_as_phenopackets(individual_list=individuals, metadata=metadata)

We output 12 GA4GH phenopackets to the directory phenopackets


# Output HPOA files

In [44]:
ingestor = PhenopacketIngestor(indir="phenopackets")
ppkt_d = ingestor.get_phenopacket_dictionary()
ppkt_list = list(ppkt_d.values())

[pyphetools] Ingested 24 GA4GH phenopackets.


In [45]:
builder = HpoaTableBuilder(phenopacket_list=ppkt_list)
hpoa_table_creator = builder.autosomal_dominant(PMID).build()

We found a total of 82 unique HPO terms
Extracted disease: Neurodevelopmental disorder with impaired language, behavioral abnormalities, and dysmorphic facies (OMIM:620494)


In [46]:
hpoa_table_creator.get_dataframe()

,#diseaseID,diseaseName,phenotypeID,phenotypeName,onsetID,onsetName,frequency,sex,negation,modifier,description,publication,evidence,biocuration
0,OMIM:620494,"Neurodevelopmental disorder with impaired language, behavioral abnormalities, and dysmorphic facies",HP:0001511,Intrauterine growth retardation,,,2/2,,,,,PMID:35977029,PCS,ORCID:0000-0002-5648-2155
1,OMIM:620494,"Neurodevelopmental disorder with impaired language, behavioral abnormalities, and dysmorphic facies",HP:0003097,Short femur,,,1/1,,,,,PMID:35977029,PCS,ORCID:0000-0002-5648-2155
2,OMIM:620494,"Neurodevelopmental disorder with impaired language, behavioral abnormalities, and dysmorphic facies",HP:0011968,Feeding difficulties,,,8/11,,,,,PMID:35977029,PCS,ORCID:0000-0002-5648-2155
3,OMIM:620494,"Neurodevelopmental disorder with impaired language, behavioral abnormalities, and dysmorphic facies",HP:0001508,Failure to thrive,,,4/8,,,,,PMID:35977029,PCS,ORCID:0000-0002-5648-2155
4,OMIM:620494,"Neurodevelopmental disorder with impaired language, behavioral abnormalities, and dysmorphic facies",HP:0000750,Delayed speech and language development,,,10/10,,,,,PMID:35977029,PCS,ORCID:0000-0002-5648-2155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,OMIM:620494,"Neurodevelopmental disorder with impaired language, behavioral abnormalities, and dysmorphic facies",HP:0011463,Childhood onset,,,5/12,,,,,PMID:35977029,PCS,ORCID:0000-0002-5648-2155
165,OMIM:620494,"Neurodevelopmental disorder with impaired language, behavioral abnormalities, and dysmorphic facies",HP:0003621,Juvenile onset,,,7/12,,,,,PMID:35977029,PCS,ORCID:0000-0002-5648-2155
166,OMIM:620494,"Neurodevelopmental disorder with impaired language, behavioral abnormalities, and dysmorphic facies",HP:0003621,Juvenile onset,,,7/12,,,,,PMID:28258187,PCS,ORCID:0000-0002-5648-2155
167,OMIM:620494,"Neurodevelopmental disorder with impaired language, behavioral abnormalities, and dysmorphic facies",HP:0011463,Childhood onset,,,5/12,,,,,PMID:28258187,PCS,ORCID:0000-0002-5648-2155


In [47]:
hpoa_table_creator.write_data_frame()

Wrote HPOA disease file to OMIM-620494.tab
